In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator
from scipy import stats
from scipy.spatial.distance import cdist #for euclidean distance of consistency
from numpy import inf
import networkx as nx
from dtaidistance import dtw
import random
from sklearn.preprocessing import normalize
import uuid #to generate random id
import pickle #to save/load list of selected hens
import cv2

#test equal variance
from scipy.stats import levene

#clustering
from sklearn.cluster import AgglomerativeClustering 

#scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#interpolate curves for clustering among birds with not exact same days tracked
#from scipy.interpolate import interp1d

#modelling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, cohen_kappa_score, r2_score,\
mean_squared_error, mean_absolute_error, explained_variance_score#catboost, for a better support of categorical data
from catboost import CatBoostClassifier, Pool
from scipy.stats import pearsonr, spearmanr 
import scipy.stats as stats

#PCA
from sklearn import decomposition

#clustering
from sklearn.cluster import KMeans #only numerical var
from sklearn.metrics import silhouette_samples, silhouette_score
import kmodes
from kmodes.kmodes import KModes #with categorical var as well

#plot
import matplotlib.pyplot as plt
import seaborn as sns
import dexplot as dxp #for barplot
from mpl_toolkits.mplot3d import Axes3D

#network
import networkx as nx

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import chi2_distance, ts_visual, time_series_henColumn_tsRow, correct_key, FB_daily
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


# Define parameters

In [3]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_dataoutput = config.path_dataoutput
path_initial_data = config.path_initial_data
id_run = config.id_run
#id_run = 'chapter0_final_'
#path_dataoutput = r'G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\DataOutput'
#path_extracted_data = os.path.join(path_dataoutput,'TrackingSystem') 
#path_extracted_data = os.path.join(path_extracted_data, id_run)
dico_night_hour = config.dico_night_hour
dico_matching = config.dico_matching
li_binmn = config.li_binmn
penalty = config.penalty
dico_HAID_date = config.dico_HAID_date
dico_window = config.dico_window
path_extracted_HA = config.path_extracted_HA
path_extracted_HA_visual = config.path_extracted_HA_visual
path_extracted_data_SNA = config.path_extracted_data_SNA
#create a director if not existing
if not os.path.exists(path_extracted_HA_visual):
    os.makedirs(path_extracted_HA_visual)
print(id_run)

change the configuration file if not done yet!
correctlightschedule_


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download variables

In [ ]:
#network var
df_SNA = pd.read_csv(os.path.join(path_extracted_data,id_run+'_df_SNA.csv'), sep=';', parse_dates=['date'], dayfirst=True) 
df_SNA.rename({'date':'level'},axis=1,inplace=True)
print(df_SNA.shape)
display(df_SNA.head(3))
li_df = []
for v in ['closeness', 'betweeness', 'degree']:
    df_n = df_SNA.pivot(index=['HenID', 'level'], columns='distance_measure', values=v).reset_index()
    df_n.rename({x:v+'_'+x for x in df_n.columns if x.startswith('DTW_')}, axis=1, inplace=True)
    li_df.append(df_n)
#only display info on the last df    
print(df_n.shape)
display(df_n.head(3))
df_SNA_daily = pd.merge(li_df[0], li_df[1], on=['HenID','level'], how='outer')
df_SNA_daily = pd.merge(df_SNA_daily, li_df[2], on=['HenID','level'], how='outer')
print(df_SNA_daily.shape)
df_SNA_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#daily variables (one row per (henID, date))
df_daily = pd.read_csv(os.path.join(path_extracted_data, 'daily_ALL_Variable_Tranformed.csv'), sep=';',
                     parse_dates=['level'], dayfirst=True) 
df_daily['DOA'] = df_daily['level'].map(lambda x: (x-dt.datetime(2020,6,3)).days) 
df_daily['weeks_in_laying_barn'] = df_daily['DOA'].map(lambda x: int((x-119)/7)+1)
#df_daily['weeks_in_laying_barn'] = df_daily['DOA'].map(lambda x: math.ceil(x/7))
#first two month seems good from the %of hens not moving plot! and sounds good too (twice longer thatn they need to start moving)
print(df_daily.shape)

#filter by date
df_daily = df_daily[df_daily['level']>dt.datetime(2020,9,29)]
print(df_daily.shape)

#remove days that are not fully recorded
df_daily['nbr_sec_per_day'] = df_daily['level'].map(lambda x: dico_night_hour[correct_key(x,dico_night_hour)]['nbr_hour']*60*60)
df_daily['is_correct_amount_time'] = df_daily.apply(lambda x: x['nbr_sec_per_day']==x['verification_daily_total_duration'], axis=1)
df_daily[(~df_daily['is_correct_amount_time'])&(~df_daily['Total_number_transition'].isnull())][['level', 'HenID', 'Total_number_transition', 'dur_values', 'verification_daily_total_duration','nbr_sec_per_day']]
display(df_daily = df_daily[~((~df_daily['is_correct_amount_time'])&(~df_daily['Total_number_transition'].isnull()))])
print(df_daily.shape)
df_daily.head(3)

In [ ]:
#focal birds info (one row per bird) for weight
df_FB = pd.read_csv(os.path.join(path_extracted_data,id_run+'df_FOCALBIRDS.csv'), sep=';', parse_dates=['InitialStartDate'],
                     dayfirst=True) 
df_FB['percentage_of_gain_weight'] = df_FB.apply(lambda x: (x['weight 23-11-2020']-x['29-09 weight'])/x['29-09 weight']*100, axis=1)
print(df_FB.shape)
df_FB.head(3)

In [ ]:
#HA - feather & leg injuries/podo/bumble
#search in any folder and creat a list of img paths
#os.walk: Python method walk() generates the file names in a directory tree by walking the tree top-down or bottom-up
#to avoid saving images several times, we will do it specifically for images in the KBF folder:
li_df = []
for path_ in glob.glob(os.path.join(path_dataoutput, 'HA', 'HA*', 'HA*.csv')):
    HAID = path_.split('\\')[-1].split('.')[0]
    print('---------------------', HAID)
    df_ = pd.read_csv(path_, sep=';') 
    df_['HAID'] = HAID
    df_['date'] = dico_HAID_date[HAID]
    df_ = df_.rename(columns={'Pen ':'Pen'})
    df_ = df_[~df_['FocalLegringName'].isnull()]
    print(df_.shape)
    #print(df_.columns)
    #display(df_.head(2))
    li_df.append(df_)
df_HA = pd.concat(li_df)
#small typo corrections
df_HA['BirdType'] = df_HA['BirdType'].fillna('normal')
df_HA['BirdType'] = df_HA['BirdType'].map(lambda x: x.strip(' '))
df_HA = df_HA.rename(columns={'Pen':'PenID'})
df_HA['PenID'] = df_HA['PenID'].map(lambda x: 'pen'+str(int(x)))
print(df_HA.columns)
print(df_HA.shape)
df_HA.head(3)

In [ ]:
#KBF
df_KBF = pd.read_csv(os.path.join(path_extracted_HA, 'KBF_ALL_RECORDS.csv'), sep=';', engine='python') 
df_KBF = df_KBF[~df_KBF['BirdID'].isnull()]
df_KBF['Info'] = df_KBF['BirdID'].map(lambda x: re.split('(\d+)',x.strip(' ')))
df_KBF['PenID'] = df_KBF['Info'].map(lambda x: 'pen'+str(int(x[3])))
df_KBF['FocalLegringName'] = df_KBF['Info'].map(lambda x: str(int(x[1]))+str(x[2]))
df_KBF['backpack'] = df_KBF['Info'].map(lambda x: x[-1])
print(df_KBF.shape)
df_KBF.head(3)

# add HenID & weight to HA

In [ ]:
######## create one dataframae with all HA and with HenID!
#open info
df_FB_daily = FB_daily(config)
print(df_FB_daily.shape)
df_FB_daily['PenID'] = df_FB_daily['PenID'].map(lambda x: 'pen'+str(int(x)))
display(df_FB_daily[['date','FocalLegringName','PenID','HenID']].head(3))
df_FB_daily['date'].iloc[0]
#remove typos
for v in ['FocalLegringName','PenID']:
    df_HA[v] = df_HA[v].map(lambda x: x.strip(' '))
    df_FB_daily[v] = df_FB_daily[v].map(lambda x: x.strip(' '))
#as epi and epi_new are exchanged during HA; we should take the date before/ respec. after HA to match. 
#to avoid any confusioon we enter them by hand (in column SomeHenID). Same for new legrings!
df_HA_HenID = pd.merge(df_HA, df_FB_daily[['date','FocalLegringName','PenID','HenID']], 
                  on=['date','FocalLegringName','PenID'], how='left') 
print(df_HA.shape, df_FB_daily.shape, df_HA_HenID.shape)
#add the manually the anotated HenID (to avoid mistakes, in the case we would change legring or have two 
#birds from same pen with same legringID (different tagID), Then we would simply annotated manually the HenID when entering 
#the HA observations)
df_HA_HenID['HenID'] = np.where(~df_HA_HenID['SomeHenID'].isnull(), df_HA_HenID['SomeHenID'], df_HA_HenID['HenID']) 
#small verification
#display(df_HA_HenID[~df_HA_HenID['SomeHenID'].isnull()][['SomeHenID','HenID']])
###small verification
#print(df_HA_HenID[df_HA_HenID['HenID'].isnull()][['date','HAID','PenID','HenID','FocalLegringName']].shape)
#df_ = df_HA_HenID[df_HA_HenID['HenID'].isnull()][['date','HAID','PenID','HenID','FocalLegringName']].groupby(['date','HAID','PenID'])['FocalLegringName'].agg(lambda x: set(x)).reset_index()
#df_['nbr_NA_legringID'] = df_['FocalLegringName'].map(lambda x: len(x))
#bird 35 black on HA1 pen 8 was excluded due to missfucntioning tag: this it doe snot match any henid, but thats normal
#print('HERE IS THE OBSERVAION DURING HA THAT DO NOT MATCH ANY HENID:')
#display(df_)
display(df_HA_HenID[df_HA_HenID['HenID'].isnull()])
df_HA_HenID = df_HA_HenID[~df_HA_HenID['HenID'].isnull()]
df_HA_HenID['HenID'] = df_HA_HenID['HenID'].map(lambda x: 'hen_'+str(int(x)))
#small typos
#for now we replace nan with the one we will select, and then we select randomly
df_HA_HenID['Person_toe'] = df_HA_HenID['Person_toe'].fillna('C')
df_HA_HenID['Person_feather'] = df_HA_HenID['Person_feather'].fillna('Sabine')
df_HA_HenID['Person_toe'] = df_HA_HenID['Person_toe'].map(lambda x: x.strip(' '))
df_HA_HenID['Person_feather'] = df_HA_HenID['Person_feather'].map(lambda x: x.strip(' '))
print(df_HA_HenID.shape)
df_HA_HenID.head(3)

In [ ]:
#display(df_HA_HenID[(df_HA_HenID['PenID']=='pen11')&(df_HA_HenID['HAID']=='HA4')&(df_HA_HenID['FocalLegringName']=='28green')][['HAID','Person_feather','Person_toe','Reliability_MC','Reliability_SS']])

In [ ]:
#keep when its a reliability only Sabine (as Sabine learned to Satar) THEN take randomly among several remaining obs.
#ATTENTION when reliability =+: we should have at least two people rating it!
df_HA_HenID['toremove'] = df_HA_HenID.apply(lambda x: (x['Reliability_SS']==1)&(x['Person_feather']=='Satar'), axis=1)
print(df_HA_HenID.shape)
df_HA_HenID = df_HA_HenID[~df_HA_HenID['toremove']]
print(df_HA_HenID.shape)
#remove random observation if there is two from reliability
random.seed(0) #replicable way
df_HA_HenID = shuffle(df_HA_HenID)
df_HA_HenID.drop_duplicates(subset=['HenID','HAID'], keep='first', inplace=True)
#merge all HA from reliability using a random obs for now (then most reliable, once we know and once we have all the name)
df_ = df_HA_HenID.groupby(['HenID','HAID'])['Person_toe','Person_feather'].agg(lambda x: list(x)).reset_index()
df_['nbr_raters'] = df_['Person_feather'].map(lambda x: len(x))
if df_[df_['nbr_raters']>1].shape[0]>0:
    print('ERROR, we still have several observation for a/some (HenID,HAID)')
    sys.exit()
print(df_HA_HenID.shape)
df_HA_HenID.head(3)

In [ ]:
#display(df_HA_HenID[(df_HA_HenID['PenID']=='pen11')&(df_HA_HenID['HAID']=='HA4')&(df_HA_HenID['FocalLegringName']=='28green')][['HAID','Person_feather','Person_toe','Reliability_MC','Reliability_SS']])

In [ ]:
#nbr of HA per hen
df_HA_HenID.groupby('HenID')['HAID'].count().reset_index().head(3)

In [ ]:
#add weight to (HenID,HAID)
dico_HAID_h_weight = {}
dico_HAID_h_weight['HA1'] = dict(zip(df_FB['HenID'].tolist(), df_FB['weight 23-11-2020'].tolist()))
dico_HAID_h_weight['HA2'] = dict(zip(df_FB['HenID'].tolist(), df_FB['weight 04-01-2021'].tolist()))
dico_HAID_h_weight['HA3'] = dict(zip(df_FB['HenID'].tolist(), df_FB['weight 01-02-21'].tolist()))
dico_HAID_h_weight['HA4'] = dict(zip(df_FB['HenID'].tolist(), df_FB['weight 12-04-21'].tolist()))
df_HA_HenID['weight'] = df_HA_HenID.apply(lambda x: dico_HAID_h_weight[x['HAID']].get(x['HenID'],np.nan), axis=1)
print('Note that if a hen is in df_HA_HenID, it also should have the weight, look at exception:')
display(df_HA_HenID[df_HA_HenID['weight'].isnull()])
df_HA_HenID.head(3)

In [ ]:
# add WOA and DOA
df_HA_HenID['DOA'] = df_HA_HenID['date'].map(lambda x: (x-dt.datetime(2020,6,3)).days)
df_HA_HenID['WOA'] = df_HA_HenID['DOA'].map(lambda x: math.ceil(x/7))
df_HA_HenID['WOA'].value_counts()

# Add KBF ratings

In [ ]:
df_HA.head(3)

In [ ]:
df_HA_KBF = pd.merge(df_HA_HenID, df_KBF, on=['HAID','PenID','FocalLegringName'], how='outer')
print(df_HA_HenID.shape, df_KBF.shape, df_HA_KBF.shape)

In [ ]:
#df_HA_KBF[(df_HA_KBF['FocalLegringName']=='9grey')&(df_HA_KBF['HAID']=='HA1')]

In [ ]:
#df_HA_HenID[(df_HA_HenID['PenID']=='pen4')&(df_HA_HenID['HAID']=='HA4')]#[['FocalLegringName', 'imageID','backpack','HAID']]

In [ ]:
#df_KBF[(df_KBF['PenID']=='pen11')&(df_KBF['HAID']=='HA4')][['FocalLegringName', 'imageID','backpack','HAID']]

In [ ]:
#HA1, HA3, HA4 for now we have only
#df_HA_KBF[df_HA_KBF['severity'].isnull()].groupby(['HAID','PenID'])[['FocalLegringName']].agg(lambda x: set(x)).reset_index()

In [ ]:
#35black from pen 8 in HA1 is removed so its normal (s tag disfunctioning during that time)
#37orange is missing in HA1 pen 9?!
#pen3 in HA4 is missing in HA: 25green, 49grey, 31grey, 14green
#df_HA_KBF[df_HA_KBF['HenID'].isnull()].groupby(['HAID','PenID'])['FocalLegringName'].agg(lambda x: set(x)).reset_index()

In [ ]:
#save
df_HA_KBF.to_csv(os.path.join(path_extracted_data,'df_all_HA.csv'), index=False, sep=';')

In [ ]:
df_HA_KBF.columns

In [ ]:
#do visual of HenID KBF over time
df_plot = df_HA_KBF.sort_values('HAID').copy()
df_plot = df_plot[~df_plot['actual_path_img'].isnull()] #missing KBF
df_plot = df_plot.groupby(['HenID'])[['HAID','actual_path_img','severity','has_gap']].agg(lambda x: list(x)).reset_index()
for i in tqdm.tqdm(range(df_plot.shape[0])):
    x = df_plot.iloc[i]
    li_path = x['actual_path_img']
    li_HAID = x['HAID']
    henid = x['HenID']
    li_sev = x['severity']
    li_gap = x['has_gap']
    c = len(li_path) ; l = 1
    fig = plt.figure(figsize=(c*3, l*3)) #/100 si trop grand nbr
    for i,path_ in enumerate(li_path):
        plt.subplot(l,c,i+1)
        plt.tight_layout() #avoid titles superpositions
        img = cv2.imread(path_)
        plt.imshow(img)
        plt.xticks([]) #remove xlabel annotations
        plt.yticks([])
        plt.title(li_HAID[i]+' sev:'+str(li_sev[i])+' gap:'+str(li_gap[i]), size=9)
    plt.savefig(os.path.join(path_extracted_HA_visual,'KBF_'+henid+'.png'),dpi=300,format='png',bbox_inches='tight')

# Add SNA

In [ ]:
#df_daily and df_SNA_daily do not have the same lenght, as sometimes we have in df_daily the nights variables, while never
#in df_SNA_daily
df = pd.merge(df_daily, df_SNA_daily, on=['HenID','level'], how='outer')
print(df_daily.shape, df_SNA_daily.shape, df.shape)
#check if we have some days with daily var in df_daily, but with no SNA
if df[(df['closeness_DTW_30_1_Zone'].isnull())&(~df['verification_daily_total_duration'].isnull())].shape[0]>0:
    print('you have not the same amount of SNA and DAILY observations (HenID, level)')
df.head(3)

In [ ]:
#remove columns with list or that was here simply to compute/verify other variables, as wew ill now not use these variables 
df = df.drop(['dur_values','dur_values_normalized','previous_dur_values_normalized', 'FirstTimestamp_1_Zone', 
              'FirstTimestamp_2_Zone', 'FirstTimestamp_3_Zone', 'FirstTimestamp_4_Zone', 'FirstTimestamp_5_Zone',
              'list_timestamps', 'list_timestamps_seondsOfTheDay','list_of_night20_2_temperature',
              'list_of_MovementCounter_day','list_of_durations','list_of_zones','t_DU_missingZone_mvtPerc', 
              'li_event_chaoticmvt_z_d'],axis=1)

In [ ]:
#save
df.to_csv(os.path.join(path_extracted_data,'daily_ALL_Variable_Tranformed_HA_SNA.csv'), index=False, sep=';')

In [ ]:
print(list(df_SNA_daily.columns))

# Playground

In [ ]:
df_HA_HenID.columns

In [ ]:
df_HA_HenID['Diarrhea'].unique()

In [ ]:
li_HAvar = ['CombPecks', 'RFPododermatitis', 'RFBumblefoot', 'RFinjuries',
            'LFPododermatitis', 'LFBumblefoot', 'LFinjuries','NeckFeather',
            'BreastFeather', 'CloacaFeather', 'BackFeather', 'WingFeather','TailFeather','weight'] #Diarrhea
df_HA_HenID[['HenID','date']+li_HAvar].head(3)

In [ ]:
df_HA_HenID['Toes'].value_counts()

In [ ]:
df_HA_HenID['Claw'].value_counts()

In [ ]:
for v in li_HAvar:
    ax = sns.boxplot(x="WOA", y=v, data=df_HA_HenID);
    plt.title(v.upper())
    plt.savefig(os.path.join(path_extracted_HA_visual,v+'.png'),dpi=300,format='png',bbox_inches='tight')
    plt.show()